## Prepare Environment

First of all, let's try to secure a decent GPU for our Colab! Unfortunately, it's becoming much harder to get access to a good GPU with the free version of Google Colab. However, with Google Colab Pro one should have no issues in being allocated a V100 or P100 GPU.

To get a GPU, click _Runtime_ -> _Change runtime type_, then change _Hardware accelerator_ from _None_ to _GPU_.

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


Next, we need to update the Unix package `ffmpeg` to version 4:

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bio

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and 
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of our model. Finally, we'll
use `gradio` to build a flashy demo of our fine-tuned model.

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-j7rj5_wy
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-j7rj5_wy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.0 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5858218 sha256=a2d78f792fe92a7b570de35d59e709e93aa0f216c7ec2ae39ae0188985088162
  Stored in directory: /tmp/pip-ephem-wheel-cache-ber4qiqb/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/) 
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!

Linking the notebook to the Hub is straightforward - it simply requires entering your 
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import hopsworks
project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5272


## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple. 
We can download and prepare the Common Voice splits in just one line of code. 

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

Since Hindi is very low-resource, we'll combine the `train` and `validation` 
splits to give approximately 8 hours of training data. We'll use the 4 hours 
of `test` data as our held-out test set:

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "lt", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "lt", split="test", use_auth_token=True)

print(common_voice)

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

...





Reading metadata...: 1225it [00:00, 31015.28it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]






Reading metadata...: 559it [00:00, 36880.69it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/lt/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 8884
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 3749
    })
})


Most ASR datasets only provide input audio samples (`audio`) and the 
corresponding transcribed text (`sentence`). Common Voice contains additional 
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 8884
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3749
    })
})


## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages: 
1) A feature extractor which pre-processes the raw audio-inputs
2) The model which performs the sequence-to-sequence mapping 
3) A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer, 
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer) 
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Hindi, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to 
specify the target language and the task. These arguments inform the 
tokenizer to prefix the language and task tokens to the start of encoded 
label sequences:

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Lithuanian", task="transcribe")

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_ 
both into a single `WhisperProcessor` class. This processor object 
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`, 
and can be used on the audio inputs and model predictions as required. 
In doing so, we only need to keep track of two objects during training: 
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Lithuanian", task="transcribe")

### Prepare Data

Let's print the first example of the Common Voice dataset to see 
what form the data is in:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/8f0bc9eaa8c897005a8bc2e5cdb80e6b1695f92ad9d818389efb6247b7be340a/common_voice_lt_23707708.mp3', 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        3.0736230e-06,  2.1043011e-06, -2.0997777e-07], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'Taip pat Turkijoje šioms katėms didelį pavojų kelia nelegali medžioklė ir gamtos užterštumas.'}


Since 
our input audio is sampled at 48kHz, we need to _downsample_ it to 
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model. 

We'll set the audio inputs to the correct sampling rate using dataset's 
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place, 
but rather signals to `datasets` to resample audio samples _on the fly_ the 
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample 
it to the desired sampling rate:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/8f0bc9eaa8c897005a8bc2e5cdb80e6b1695f92ad9d818389efb6247b7be340a/common_voice_lt_23707708.mp3', 'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 4.7565336e-06,
       4.1286721e-06, 2.3791606e-06], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'Taip pat Turkijoje šioms katėms didelį pavojų kelia nelegali medžioklė ir gamtos užterštumas.'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

#0:   0%|          | 0/4442 [00:00<?, ?ex/s]

#1:   0%|          | 0/4442 [00:00<?, ?ex/s]

#1:   0%|          | 0/1874 [00:00<?, ?ex/s]

#0:   0%|          | 0/1875 [00:00<?, ?ex/s]

In [ ]:
help(common_voice)

In [ ]:
common_voice.save_to_disk("common_voice")

In [ ]:
cc = DatasetDict.load_from_disk("common_voice")

In [ ]:
import os
print(os.getcwd())
print(os.listdir("./common_voice/"))
print(os.listdir("./common_voice/train"))
print(os.listdir("./common_voice/test"))

/content
['train', 'dataset_dict.json', 'test']
['dataset.arrow', 'dataset_info.json', 'state.json']
['dataset.arrow', 'dataset_info.json', 'state.json']


In [ ]:
def get_dir_size(path='/common_voice/train'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total
sz = get_dir_size(path="/content/common_voice/")
print(sz)

12134707653


In [ ]:
# upload dataset to hopsworks
dataset_api = project.get_dataset_api()

# created before in Hopsworks
# common_voice/train
# common_voice/test

In [ ]:
# Upload all of the local files to the common_voice directory you created in Hopsworks

uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/dataset_dict.json", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/", overwrite=True)

Uploading: 0.000%|          | 0/29 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/train/dataset_info.json", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/train/", overwrite=True)

Uploading: 0.000%|          | 0/1143 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/test/dataset_info.json", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/test/", overwrite=True)

Uploading: 0.000%|          | 0/1143 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/train/state.json", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/train/", overwrite=True)

Uploading: 0.000%|          | 0/289 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/test/state.json", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/test/", overwrite=True)

Uploading: 0.000%|          | 0/289 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/train/dataset.arrow", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/train/", overwrite=True)

Uploading: 0.000%|          | 0/8533580664 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "./common_voice/test/dataset.arrow", 
    upload_path = "KTH_lab1_Training_Datasets/common_voice/test/", overwrite=True)

Uploading: 0.000%|          | 0/3601124096 elapsed<00:00 remaining<?